In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import random 
from matplotlib import pyplot as plt

import os
import cv2
import glob
import shutil
import imutils
from pathlib import Path
from distutils.dir_util import copy_tree

In [2]:
img_height = 88
img_width = 128

num_classes = 124
train_data_dir = './training_data/'

epochs = 100
batch_size = 64
val_split = 0.2

f1 = 40; k1 = 3; s1 = 2
f2 = 32; k2 = 5; s2 = 3
drop = 0.2

In [3]:
def get_id(subject):
    sub = str(subject)
    if(len(sub)==1): sub = '00' + sub
    elif(len(sub)==2): sub = '0' + sub
    return sub

subjects = 124
subject_ids = [get_id(i) for i in range(1, subjects+1)] 
conditions = ['bg-01', 'bg-02', 'cl-01', 'cl-02', 'nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05', 'nm-06']
views = ['000', '018', '036', '054', '072', '090', '108', '126', '144', '162', '180']

paths = []
for subject in subject_ids:
    for condition in conditions:
        for view in views:
            path = subject + '/' + condition + '/' + view + '/'
            paths.append(path)


In [4]:
files = glob.glob('./training_data/*')
for f in files:
    shutil.rmtree(f)

for sub in subject_ids:
    os.mkdir('./training_data/%s'%sub)
    for i in range(1,5):
       files = glob.glob('./GEnIs/%s/*nm-0%s*'%(sub, i))
       for file in files:
        shutil.copy(file, './training_data/%s/'%sub)

In [ ]:
files = glob.glob('./testing_data/*')
for f in files:
    shutil.rmtree(f)

for sub in subject_ids:
    os.mkdir('./testing_data/%s'%sub)
    for i in range(5,7):
       files = glob.glob('./GEnIs/%s/*nm-0%s*'%(sub, i))
       for file in files:
        shutil.copy(file, './testing_data/%s/'%sub)

In [26]:
# 6 random sequence in training
# remaining 4 in testing
# for each subject per angle
files = glob.glob('./training_data/*')
for f in files:
    shutil.rmtree(f)
  
files = glob.glob('./testing_data/*')
for f in files:
    shutil.rmtree(f)

for sub in subject_ids:
    os.mkdir('./training_data/%s'%sub)
    os.mkdir('./testing_data/%s'%sub)
    c = []
    for i in range(0, 6):
       t = random.choice(conditions)
       while t in c:
         t = random.choice(conditions)
       c.append(t)
    
    for i in c:
       files = glob.glob('./GEnIs/%s/*%s*'%(sub, i))
       for file in files:
        shutil.copy(file, './training_data/%s/'%sub)
    
    for i in conditions:
       if i not in c:
          files = glob.glob('./GEnIs/%s/*%s*'%(sub, i))
          for file in files:
            shutil.copy(file, './testing_data/%s/'%sub)
               

In [4]:
files = glob.glob('./training_data/*')
for f in files:
    shutil.rmtree(f)

for sub in subject_ids:
    os.mkdir('./training_data/%s'%sub)
    
    for i in ['nm-01', 'nm-02', 'nm-03', 'nm-04']:
        files = glob.glob('./GEnIs/%s/*%s*'%(sub, i))
        for file in files:
         shutil.copy(file, './training_data/%s/'%sub)

In [17]:
# all the sequences of angle 90 in training
# all other angles in testing

# files = glob.glob('./training_data/*')
# for f in files:
#     shutil.rmtree(f)

files = glob.glob('./testing_data/*')
for f in files:
    shutil.rmtree(f)

for sub in subject_ids:
    # os.mkdir('./training_data/%s'%sub)
    os.mkdir('./testing_data/%s'%sub)
    # files = glob.glob('./GEnIs/%s/*%s*'%(sub, '090'))
    # for file in files:
    #     shutil.copy(file, './training_data/%s/'%sub)
    
    for v in ['126']:
        files = glob.glob('./GEnIs/%s/*%s*'%(sub, v))
        for file in files:
            shutil.copy(file, './testing_data/%s/'%sub)


In [5]:
# building data 
train_data = tf.keras.utils.image_dataset_from_directory(
  train_data_dir,
  seed=114,
  subset='training',
  batch_size=batch_size,
  validation_split=val_split,
  image_size=(img_height, img_width),
)

val_data = tf.keras.utils.image_dataset_from_directory(
  train_data_dir,
  seed=114,
  subset="validation",
  batch_size=batch_size,
  validation_split=val_split,
  image_size=(img_height, img_width),
)

Found 15876 files belonging to 124 classes.
Using 12701 files for training.
Found 15876 files belonging to 124 classes.
Using 3175 files for validation.


In [21]:
# val_batches = tf.data.experimental.cardinality(val_data)
# test_data = val_data.take((2*val_batches) // 3)
# val_data = val_data.skip((2*val_batches) // 3)

In [6]:
# cache method will help to cache the data during the first epoch
# prefetch method basically loads the next batch of data while the current batch is being processed.
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
# create the model
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
    layers.Conv2D(filters=f1, kernel_size=k1, strides=s1, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.Dropout(drop),

    layers.Conv2D(filters=f2, kernel_size=k2, strides=s2, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Dropout(drop),


    layers.Flatten(),

    layers.Dense(1024, use_bias=False),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(drop),

    layers.Dense(num_classes, activation='softmax')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 88, 128, 3)        0         
                                                                 
 conv2d (Conv2D)             (None, 44, 64, 40)        1120      
                                                                 
 batch_normalization (BatchN  (None, 44, 64, 40)       160       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 32, 40)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 32, 40)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 11, 32)         3

In [9]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

Epoch 1/100
199/199 [==============================] - 40s 192ms/step - loss: 3.9664 - accuracy: 0.1358 - val_loss: 6.6623 - val_accuracy: 0.0110
Epoch 2/100
199/199 [==============================] - 47s 235ms/step - loss: 2.0852 - accuracy: 0.4533 - val_loss: 5.7350 - val_accuracy: 0.0595
Epoch 3/100
199/199 [==============================] - 34s 172ms/step - loss: 1.2928 - accuracy: 0.6453 - val_loss: 1.7734 - val_accuracy: 0.5178
Epoch 4/100
199/199 [==============================] - 33s 164ms/step - loss: 0.9064 - accuracy: 0.7481 - val_loss: 0.6747 - val_accuracy: 0.8255
Epoch 5/100
199/199 [==============================] - 37s 188ms/step - loss: 0.6951 - accuracy: 0.8042 - val_loss: 0.4627 - val_accuracy: 0.8813
Epoch 6/100
199/199 [==============================] - 42s 210ms/step - loss: 0.5487 - accuracy: 0.8425 - val_loss: 0.3691 - val_accuracy: 0.9033
Epoch 7/100
199/199 [==============================] - 43s 216ms/step - loss: 0.4564 - accuracy: 0.8700 - val_loss: 0.3319 -

In [10]:
def test():
  test_data = tf.keras.utils.image_dataset_from_directory(
    './testing_data',
    seed=114,
    batch_size=batch_size,
    image_size=(img_height, img_width),
  )

  model.evaluate(test_data)

In [12]:
for v in views:

  # remove current data from training folder
  files = glob.glob('./testing_data/*')
  for f in files:
      shutil.rmtree(f)

  for sub in subject_ids:
    os.mkdir('./testing_data/%s'%sub)
    for i in ['nm-05', 'nm-06']:
          files = glob.glob('./GEnIs/%s/%s*%s*'%(sub, i, v))
          for file in files:
            shutil.copy(file, './testing_data/%s/'%sub)
  test()
  

Found 888 files belonging to 124 classes.
14/14 [==============================] - 1s 37ms/step - loss: 4.3796 - accuracy: 0.5315
Found 918 files belonging to 124 classes.
15/15 [==============================] - 1s 35ms/step - loss: 5.0821 - accuracy: 0.5076
Found 896 files belonging to 124 classes.
14/14 [==============================] - 1s 40ms/step - loss: 5.5623 - accuracy: 0.4632
Found 792 files belonging to 124 classes.
13/13 [==============================] - 1s 36ms/step - loss: 6.7786 - accuracy: 0.3712
Found 520 files belonging to 124 classes.
9/9 [==============================] - 0s 32ms/step - loss: 8.3788 - accuracy: 0.3712
Found 497 files belonging to 124 classes.
8/8 [==============================] - 0s 34ms/step - loss: 10.6080 - accuracy: 0.2777
Found 525 files belonging to 124 classes.
9/9 [==============================] - 0s 31ms/step - loss: 10.2485 - accuracy: 0.3010
Found 670 files belonging to 124 classes.
11/11 [==============================] - 1s 35ms/ste